In [10]:
!pip install requests
!pip install beautifulsoup4
!pip install boto3 pandas



StatementMeta(, 5e80e3a7-30cf-4526-b896-108b04bee47d, 12, Finished, Available, Finished)

In [11]:
import requests
from bs4 import BeautifulSoup
import pandas as pd


StatementMeta(, 5e80e3a7-30cf-4526-b896-108b04bee47d, 13, Finished, Available, Finished)

**Python script to webscrape medal table from the web**

In [12]:
# List of URLs to scrape
urls = [
    {"url": "https://www.bbc.com/sport/olympics/paris-2024/medals"}
    # Add more URLs as needed
]
# Create an empty list to store DataFrames
dfs = []

for url_info in urls:
    url = url_info["url"]

    # Send HTTP Request and Parse HTML
    r = requests.get(url)
    soup = BeautifulSoup(r.text, "lxml")
    tables = soup.find_all("table", class_="ssrcss-jsg8ev-TableWrapper e1xoxfm62")

    # Find and Extract Table Headers
for table in tables:
    # Find and extract table headers
    headers = table.find_all("th")
    titles = [i.text.strip() for i in headers]

    # Create an Empty DataFrame
    df = pd.DataFrame(columns=titles)

    # Iterate Through Table Rows and Extract Data
    rows = table.find_all("tr")
    for i in rows[1:]:
        data = i.find_all("td")
        row = [tr.text.strip() for tr in data]  # Apply .strip() to remove \n
        l = len(df)
        df.loc[l] = row


    # Append the DataFrame to the list
    dfs.append(df)

# Concatenate all DataFrames into a single DataFrame
olympic_standing = pd.concat(dfs, ignore_index=True)

csv_file_path = "/lakehouse/default/Files/olympic_standing.csv"
olympic_standing.to_csv(csv_file_path, index=False)
print(olympic_standing)


StatementMeta(, 5e80e3a7-30cf-4526-b896-108b04bee47d, 14, Finished, Available, Finished)

    Rank                     Country GGold SSilver BBronze TotalTotal
0      1             USUnited States    40      44      42        126
1      2                    CHNChina    40      27      24         91
2      3                    JPNJapan    20      12      13         45
3      4                AUSAustralia    18      19      16         53
4      5                   FRAFrance    16      26      22         64
..   ...                         ...   ...     ...     ...        ...
200   92                  VIEVietnam     0       0       0          0
201   92   BVIBritish Virgin Islands     0       0       0          0
202   92  AVIAmerican Virgin Islands     0       0       0          0
203   92                    YEMYemen     0       0       0          0
204   92                 ZIMZimbabwe     0       0       0          0

[205 rows x 6 columns]


**We used Data Wrangler to clean Webscraped data**
- We cleaned duplication in the column headers
- We splitted the country column to remove abbreviations from the country name

In [13]:
# Code generated by Data Wrangler for pandas DataFrame

def clean_data(olympic_standing):
    olympic_standing= olympic_standing.rename(columns={'GGold': 'Gold', 'SSilver': 'Silver','BBronze':'Bronze', 'TotalTotal': 'Total'})
    olympic_standing['Country']=olympic_standing['Country'].str[3:]
    # Replace all instances of "" with "" in column: 'Country'
    olympic_standing['Country'] = olympic_standing['Country'].str.replace("nited States", "United States", case=False, regex=False)
    # Replace all instances of "" with "" in column: 'Country'
    olympic_standing['Country'] = olympic_standing['Country'].str.replace("reat Britain", "Great Britain", case=False, regex=False)
    # Replace all instances of "" with "" in column: 'Country'
    olympic_standing['Country'] = olympic_standing['Country'].str.replace("ew Zealand", "New Zealand", case=False, regex=False)
    # Replace all instances of "" with "" in column: 'Country'
    olympic_standing['Country'] = olympic_standing['Country'].str.replace("ong Kong", "Hong Kong", case=False, regex=False)
    # Replace all instances of "" with "" in column: 'Country'
    olympic_standing['Country'] = olympic_standing['Country'].str.replace("out Africa", "South African", case=False, regex=False)
    # Replace all instances of "" with "" in column: 'Country'
    olympic_standing['Country'] = olympic_standing['Country'].str.replace("outh Africa", "South Africa", case=False, regex=False)
    return olympic_standing

olympic_standing_clean = clean_data(olympic_standing.copy())
olympic_standing_clean.head()

StatementMeta(, 5e80e3a7-30cf-4526-b896-108b04bee47d, 15, Finished, Available, Finished)

,Rank,Country,Gold,Silver,Bronze,Total
0,1,United States,40,44,42,126
1,2,China,40,27,24,91
2,3,Japan,20,12,13,45
3,4,Australia,18,19,16,53
4,5,France,16,26,22,64


In [14]:
csv_file_path = "/lakehouse/default/Files/olympic_standing_clean.csv"
olympic_standing_clean.to_csv(csv_file_path, index=False)
print(olympic_standing_clean)

StatementMeta(, 5e80e3a7-30cf-4526-b896-108b04bee47d, 16, Finished, Available, Finished)

    Rank                  Country Gold Silver Bronze Total
0      1            United States   40     44     42   126
1      2                    China   40     27     24    91
2      3                    Japan   20     12     13    45
3      4                Australia   18     19     16    53
4      5                   France   16     26     22    64
..   ...                      ...  ...    ...    ...   ...
200   92                  Vietnam    0      0      0     0
201   92   British Virgin Islands    0      0      0     0
202   92  American Virgin Islands    0      0      0     0
203   92                    Yemen    0      0      0     0
204   92                 Zimbabwe    0      0      0     0

[205 rows x 6 columns]


**Extract Countries,continent,country flag from wikipedia**

In [15]:
# URL of the JSON data
url = 'https://gist.githubusercontent.com/amitjambusaria/b9adebcb4f256eae3dfa64dc9f1cc2ef/raw/6431d72439c8399b05d2b8e9d51153e5dee7ad3b/countries.json'

# Fetch the JSON data from the URL
response = requests.get(url)
json_data = response.json()

# Convert JSON data to a DataFrame
df = pd.json_normalize(json_data)

# Selecting only required columns
country=df[['name','region']]

#rename continent abbr
country['region'] = country['region'].replace({
    'AS': 'Asia',
    'EU': 'Europe',
    'AF': 'Africa',
    'OC': 'Oceania',
    'NA': 'North America',
    'SA': 'South America'
})
print(country)

StatementMeta(, 5e80e3a7-30cf-4526-b896-108b04bee47d, 17, Finished, Available, Finished)

                  name   region
0          Afghanistan     Asia
1        Åland Islands   Europe
2              Albania   Europe
3              Algeria   Africa
4       American Samoa  Oceania
..                 ...      ...
244  Wallis and Futuna  Oceania
245     Western Sahara   Africa
246              Yemen     Asia
247             Zambia   Africa
248           Zimbabwe   Africa

[249 rows x 2 columns]


/tmp/ipykernel_5969/3127973928.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  country['region'] = country['region'].replace({


**Write to File**

In [16]:
csv_file_path = "/lakehouse/default/Files/country.csv"
country.to_csv(csv_file_path, index=False)
#print(olympic_standing_clean)

StatementMeta(, 5e80e3a7-30cf-4526-b896-108b04bee47d, 18, Finished, Available, Finished)

**Extract flag url to be used in PowerBI**

In [17]:
# URL of the JSON data
url = 'https://cdn.jsdelivr.net/npm/country-flag-emoji-json@2.0.0/dist/by-code.json'

# Fetch the JSON data from the URL
response = requests.get(url)
json_data = response.json()

# Convert JSON data to a list of dictionaries for each country
country_list = [{'code': code, **details} for code, details in json_data.items()]

# Create a DataFrame from the list of country dictionaries
df = pd.DataFrame(country_list)

# Set Pandas display options to ensure full width of columns is shown
pd.set_option('display.max_colwidth', None)

flag = df[['name','image']]
print(flag)

StatementMeta(, 5e80e3a7-30cf-4526-b896-108b04bee47d, 19, Finished, Available, Finished)

                     name  \
0        Ascension Island   
1                 Andorra   
2    United Arab Emirates   
3             Afghanistan   
4       Antigua & Barbuda   
..                    ...   
256                Zambia   
257              Zimbabwe   
258               England   
259              Scotland   
260                 Wales   

                                                                                   image  
0          https://cdn.jsdelivr.net/npm/country-flag-emoji-json@2.0.0/dist/images/AC.svg  
1          https://cdn.jsdelivr.net/npm/country-flag-emoji-json@2.0.0/dist/images/AD.svg  
2          https://cdn.jsdelivr.net/npm/country-flag-emoji-json@2.0.0/dist/images/AE.svg  
3          https://cdn.jsdelivr.net/npm/country-flag-emoji-json@2.0.0/dist/images/AF.svg  
4          https://cdn.jsdelivr.net/npm/country-flag-emoji-json@2.0.0/dist/images/AG.svg  
..                                                                                   ...  
256        htt

**Write to File**

In [19]:
csv_file_path = "/lakehouse/default/Files/flag.csv"
flag.to_csv(csv_file_path, index=False)
#print(olympic_standing_clean)

StatementMeta(, 5e80e3a7-30cf-4526-b896-108b04bee47d, 21, Finished, Available, Finished)